## **1. Importación de librerías**

In [ ]:
# Tratamiento de datos.

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## **2. Carga de datos `dataset_estudiantes.csv`**

Se importa el fichero `dataset_estudiantes.csv` en un DataFrame.

In [2]:
df = pd.read_csv("../data/1.raw/dataset_estudiantes.csv")

## **3. Definición de objetivos y variables predictoras**

In [ ]:
y_regresion = df["nota_final"]
y_clasificacion = df["aprobado"]       

X = df.drop(columns=["nota_final", "aprobado"]) 

En este punto se separa el problema en sus elementos principales, definiendo por un lado las variables objetivo y por otro las variables predictoras. `y_regresion` establece la variable objetivo para la tarea de regresión, mientras que `y_clasificacion` define la variable objetivo para la tarea de clasificación. Por su parte, `X` construye el conjunto de variables predictoras eliminando las columnas objetivo, lo que evita que se produzca una fuga de información y asegura que los modelos aprendan únicamente a partir de las características de entrada.

## **4. Definición de columnas numéricas y categóricas**

In [ ]:
columns_num = ["horas_estudio_semanal", "nota_anterior", "tasa_asistencia", "horas_sueno", "edad"]

columns_cat = ["nivel_dificultad", "tiene_tutor", "horario_estudio_preferido", "estilo_aprendizaje"]

En este paso se especifica qué columnas de `X` son numéricas y cuáles son categóricas, ya que cada tipo requiere un preprocesamiento distinto. Las columnas numéricas se preparan para aplicar imputación por mediana y escalado, mientras que las columnas categóricas se preparan para imputar valores nulos y posteriormente codificarlas mediante One-Hot Encoding. Esta separación permite aplicar a cada grupo de variables la transformación adecuada dentro del pipeline.

## **5. Tratamiento de valores nulos**

En este punto se define cómo tratar los valores nulos antes de entrenar los modelos. En el conjunto de datos se han detectado nulos en dos variables categóricas `horario_estudio_preferido` y `estilo_aprendizaje`, y en la variable numérica `horas_sueno`.

Para evitar fuga de información y mantener un flujo reproducible, la imputación no se realiza modificando el DataFrame directamente, sino dentro de un `Pipeline`, de forma que los parámetros de imputación se calculen únicamente con el conjunto de entrenamiento para que se apliquen después tanto a train como a test.

En las variables numéricas, se decide imputar los valores nulos con la mediana de cada columna, ya que esta medida es robusta frente a posibles valores extremos y representa de forma estable el valor central, en el caso de `horas_sueno` la distribución observada en el EDA es relativamente concentrada, por lo que esta elección resulta la más adecuada.

En las variables categóricas, los valores nulos se imputarán con la categoría constante `Desconocido`. Con ello se mantiene explícita la ausencia del dato y se evita sustituir sistemáticamente por la categoría más frecuente. Además, esta opción facilita el modelado al tratar los nulos como una categoría adicional durante el One-Hot Encoding.”

Estas decisiones se implementarán mediante `SimpleImputer` dentro del preprocesamiento, junto con la codificación de variables categóricas, asegurando la consistencia y la reproducibilidad en todo el flujo de entrenamiento y validación.

## **6. Creación del pipeline de variables numéricas**

In [ ]:
transform_num = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])

Para las variables numéricas se define un pipeline que aplica dos operaciones en orden. Primero se imputan los valores nulos utilizando la mediana de cada columna, lo que permite tratar adecuadamente los datos ausentes de forma robusta. Después, se realiza el escalado mediante `StandardScaler` para que todas las variables numéricas queden en una escala comparable, algo muy útil para modelos más sensibles a la magnitud de las variables. En modelos basados en árboles este paso no es imprescindible, pero se mantiene para garantizar un preprocesamiento consistente.

## **7. Creación del pipeline de variables categóricas**

In [ ]:
transform_cat = Pipeline(steps=[("imputer", SimpleImputer(strategy="constant", fill_value="Desconocido")),("onehot", OneHotEncoder(handle_unknown="ignore"))])

Para las variables categóricas se construye un pipeline específico que, primero imputa los valores nulos con una categoría constante "Desconocido" para conservar la información ausente de las variables. A continuación, se aplica `OneHotEncoder` para convertir las categorías en variables binarias, permitiendo que los modelos trabajen con estas columnas. Además, se configura para que ignore categorías no vistas durante el entrenamiento, evitando de esta forma errores al transformar el conjunto de validación o test.

## **8. Unión de ambos pipelines con ColumnTransformer** 

In [ ]:
preprocess = ColumnTransformer(transformers=[("num", transform_num, columns_num), ("cat", transform_cat, columns_cat)])

En este paso se unifican los dos pipelines definidos previamente mediante ColumnTransformer, con el objetivo de aplicar transformaciones diferentes según el tipo de variable. De esta forma, el pipeline numérico se aplica únicamente a las columnas numéricas para imputar nulos y escalar, mientras que el pipeline categórico se aplica solo a las columnas categóricas para imputar valores nulos y realizar la codificación One-Hot Encoding. El resultado es un preprocesador único y reutilizable que transforma `X` en una matriz totalmente numérica y lista para el entrenamiento de modelos en las fases posteriores.

## **9. Comprobación del preprocesamiento**

In [8]:
X_transformed = preprocess.fit_transform(X)

print("Shape X original:", X.shape)
print("Shape X transformada:", X_transformed.shape)

Shape X original: (1000, 9)
Shape X transformada: (1000, 19)


La matriz original tiene 1000 registros y 9 variables predictoras. Tras aplicar el preprocesamiento, el número de filas se mantiene en 1000, pero el número de columnas aumenta a 19 debido al One-Hot Encoding de las variables categóricas, que genera columnas binarias para cada categoría. Esta transformación se realiza únicamente como una comprobación técnica de dimensiones. En las fases 3 y 4, el preprocesamiento se ajustará exclusivamente con el conjunto de entrenamiento dentro de un Pipeline, evitando fuga de información y asegurando una evaluación fiable en el conjunto de prueba.

## **10. Preparación del Split**

In [33]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_regresion, test_size=0.2, random_state=42)

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y_clasificacion, test_size=0.2, random_state=42, stratify=y_clasificacion)

En este paso se realiza la partición del conjunto de datos en entrenamiento y prueba, con el objetivo de evaluar los modelos sobre datos no vistos y obtener una estimación lo más realista posible de su rendimiento. Se generan dos divisiones independientes, una para regresión con `nota_final` como variable objetivo y otra para clasificación con `aprobado`. En el caso de clasificación se utiliza partición estratificada, de modo que se mantiene en train y test la misma proporción de aprobados y suspensos observada en el conjunto original, lo cual es especialmente relevante debido al desbalance de clases detectado en el EDA.

### **Comprobación final**

In [31]:
print("Regresión:", X_train_reg.shape, X_test_reg.shape, "\n")
print("Clasificación:", X_train_clf.shape, X_test_clf.shape, "\n")
print("% aprobado original:", y_clasificacion.mean() * 100, "%")
print("% aprobado train:", y_train_clf.mean() * 100,"%" )
print("% aprobado test:", y_test_clf.mean() * 100, "%")

Regresión: (800, 9) (200, 9) 

Clasificación: (800, 9) (200, 9) 

% aprobado original: 89.8 %
% aprobado train: 89.75 %
% aprobado test: 90.0 %


En esta comprobación final se verifica que el preprocesamiento y la partición de datos se han realizado correctamente. Tras aplicar el ColumnTransformer, el conjunto `X` mantiene los 1000 registros, pero pasa de 9 a 19 columnas debido a la codificación one-hot de las variables categóricas, que genera nuevas variables binarias.

Posteriormente, se comprueba la división en entrenamiento y prueba, tanto en regresión como en clasificación se obtienen 800 muestras para entrenamiento y 200 para test, manteniendo las mismas 9 variables predictoras antes de transformar. Por último, se valida que la partición estratificada en clasificación conserva la proporción de la clase aprobado de forma consistente, ya que el porcentaje original es 89,8% y se mantiene prácticamente igual en train 89,75% y en test 90%, lo que asegura una evaluación más representativa del modelo.